In [1]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

import pyopencl as cl
import numpy as np
import scipy.interpolate as intrp
from scipy import stats
import time
import matplotlib.pyplot as plt
import patsy # for comparison

In [2]:
def compute_multiplicand_for_beta(X, weights = 1, df = 1):
  """
  Computes inv(XtWX+lambdaI), which can then give us 
  beta via the simple inv(XtWX+lambdaI)@XtWg
  """
  X_tilde = (X.T*np.sqrt(weights)).T
  # QR decompose X
  R = np.linalg.qr(X_tilde)[1]
  # Find smoothing parameter
  if X.shape[1] <= df:
    k = 0
  else:
    inv_eig =  np.linalg.eigvalsh(R @ R.T)
    k = 0.000001 # Any small value would do
    # Find Newton-Rhapson step
    def find_step(k):
      h = 1+k/inv_eig # for performance
      fx = np.sum(1/h)-1
      dfdx = np.sum(-1/(inv_eig * (h**2)))
      delta = -fx/dfdx
      return delta
    
    # Find root
    delta = 3. # Any large value would do
    while delta > 1e-7: 
      delta = find_step(k)
      k += delta

  XtWX = R.T @ R
  # add lambdaI
  d = np.einsum('ii->i', XtWX)
  d += k
  # Final result
  return np.linalg.inv(XtWX)

def roundUp(numToRound, multiple):
    if (multiple == 0):
        return numToRound
    remainder = numToRound % multiple
    if (remainder == 0):
        return numToRound
    return numToRound + multiple - remainder

In [3]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

In [4]:
n_rows = 1000
n_learners = 100
n_effects = 2
w = np.ones(n_rows, dtype = np.float32)
# xp.random.multinomial(n, [1/n]*n, size=1).reshape((n, ))

#---- Generate data
# Add sorting if you want to plot one of the functions
independent_variables = \
  [np.random.default_rng(i).uniform(0, 1, n_rows) for i in range(n_learners)]

# Simulate f(x) = 7 + 10*sin(2 pi x) 
intercept = 7

mu = intercept

effect_index = np.random.default_rng(111).integers(
    0, n_learners, size = n_effects, 
    dtype = np.int32)
for i in effect_index:
  mu += 10*np.sin(independent_variables[i]*2*np.pi) + independent_variables[i]

epsilon = np.random.default_rng(777).normal(size = n_rows, scale = .001)
y = mu + epsilon
effect_index


array([47, 15])

In [5]:
#---- Compute basis matrices
raw_design_matrices = \
  [patsy.dmatrix(
      "bs(x, df=32, degree=3, include_intercept=True)-1", 
      {"x": x}) for x in independent_variables]

# Centre
center_function = lambda x: x - x.mean()
design_matrices = \
  [np.apply_along_axis(center_function, 0, B).astype(np.float32) for B in raw_design_matrices]

# B_center = np.asarray(B_center).astype(np.float32)
# # y_center = (y - np.mean(y, axis=0))
# # plt.subplot(1,2,1)

# plt.plot(X[0], y)
# plt.title('B-spline basis')

multiplicands_for_beta = [
    compute_multiplicand_for_beta(X, weights = w) for X in design_matrices
    ]

In [13]:
gpu_matmul = """
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#ifndef USE_DOUBLE
#define USE_DOUBLE 0
#endif

#if USE_DOUBLE
#pragma OPENCL EXTENSION cl_khr_fp64 : enable
typedef double scalar_t;
#else
typedef float scalar_t;
#endif

#define BLOCK_SIZE 16
#define global_idx(x_idx, y_idx, m) (x_idx * m + y_idx)

#define WPT 8
#define RBLOCK_SIZE (BLOCK_SIZE/WPT)

#define ROW_DIM 0
#define COL_DIM 1

// -- Uses NDRange Kernel with Local Memory and 2D Register tiling
// M, N, P can be arbitrary sizes
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE / WPT, BLOCK_SIZE / WPT, 1)))
void matmul(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P,
      __const int M_,
      __const int N_,
      __const int P_)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float Areg;
    float Breg[WPT];
    float acc[WPT][WPT];
    for(int wm=0; wm<WPT; wm++){
        for(int wn=0; wn<WPT; wn++){
            acc[wm][wn] = 0.0f;
        }
    }
    const int numTiles = N_/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int wm=0; wm<WPT; wm++){
            for (int wn=0; wn<WPT; wn++){
                const int r = BLOCK_SIZE*t + row;
                const int c = BLOCK_SIZE*t + col;
                if(((m + wm*RBLOCK_SIZE) < M) && ((c + wn*RBLOCK_SIZE) < N)){
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = A[(m + wm*RBLOCK_SIZE)*N + (c + wn*RBLOCK_SIZE)];
                } else {
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }

                if(((p + wn*RBLOCK_SIZE) < P) && ((r + wm*RBLOCK_SIZE) < N)){
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = B[(r + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)];
                } else {
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int wn=0; wn<WPT; wn++){
                Breg[wn] = B_local[k][col + wn*RBLOCK_SIZE];
            }
            for (int wm=0; wm<WPT; wm++){
                Areg = A_local[row + wm*RBLOCK_SIZE][k];
                for (int wn=0; wn<WPT; wn++){
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int wm=0; wm<WPT; wm++){
        for (int wn=0; wn<WPT; wn++){
            if(((m + wm*RBLOCK_SIZE) < M) && ((p + wn*RBLOCK_SIZE) < P)){
                C[(m + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)] = acc[wm][wn];
            }
            
        }
    }
}

__kernel void matmul_global(__global float* C, 
          __global float* A, 
          __global float* B, 
          const int wA, const int wB){
  
   int tile_x = get_global_id(0); 
   int tile_y = get_global_id(1);
 
   // value stores the element that is computed by the thread
   float value = 0;
   for (int k = 0; k < wA; ++k){
      float elementA = A[tile_y * wA + k];
      float elementB = B[k * wB + tile_x];
      value += elementA * elementB;

      barrier(CLK_LOCAL_MEM_FENCE);
   }
 
   // Write the matrix to device memory each thread writes one element
   C[tile_y * wA + tile_x] = value;
}

__kernel void matvecmul(int M,
                        int N,
                        const global float *A,
                        const global float *x,
                        global float *y)
    {   
        int i = get_global_id(0);
        float acc = 0.0f;
        for (int j=0; j<N; j++)
        {
            acc += A[M * j + i] * x[j];
        }
        y[i] = acc;
}

__kernel void gemv1(int m, int n,__global const scalar_t * a,__global const scalar_t * x,
                    __global scalar_t * y){
    scalar_t sum = 0.0f;
    int i = get_global_id(0); // row index
    for (int k=0;k<n;k++)
        {
        sum += a[i + m*k] * x[k];
        }
    y[i] = sum;
}

// GEMV2
__kernel void gemv2(int m,int n, __global const float * a,
                    __global const float * x,
                    __global float * y
                    ){

  // Compute partial dot product
  __local float work[16];
  float sum = 0;
  for (int k=get_global_id(COL_DIM);k<n;k+=get_global_size(COL_DIM))
    {
      sum += a[get_global_id(ROW_DIM)+m*k] * x[k];
    }

  // Each thread stores its partial sum in WORK
  int rows = get_local_size(ROW_DIM); // rows in group
  int cols = get_local_size(COL_DIM); // initial cols in group
  int ii = get_local_id(ROW_DIM); // local row index in group, 0<=ii<rows
  int jj = get_local_id(COL_DIM); // block index in column, 0<=jj<cols
  work[ii+rows*jj] = sum;
  barrier(CLK_LOCAL_MEM_FENCE); // sync group

  // Reduce sums in log2(cols) steps
  while ( cols > 1 )
    {
      cols >>= 1;
      if (jj < cols) work[ii+rows*jj] += work[ii+rows*(jj+cols)];
      barrier(CLK_LOCAL_MEM_FENCE); // sync group
    }

  // Write final result in Y
  if ( jj == 0 ) y[get_global_id(ROW_DIM)] = work[ii];
}

// GEMV3
__kernel void gemv3(int m,int n, __global const float * a,
                    __global const float * x,
                    __global float * y
                    ){

  // Compute partial dot product
  __local float work[16];
  float sum = 0;
  for (int k=get_global_id(COL_DIM);k<n;k+=get_global_size(COL_DIM))
    {
      sum += a[get_global_id(ROW_DIM)+m*k] * x[k];
    }

  // Each thread stores its partial sum in WORK
  int rows = get_local_size(ROW_DIM); // rows in group
  int cols = get_local_size(COL_DIM); // initial cols in group
  int ii = get_local_id(ROW_DIM); // local row index in group, 0<=ii<rows
  int jj = get_local_id(COL_DIM); // block index in column, 0<=jj<cols
  work[ii+rows*jj] = sum;
  barrier(CLK_LOCAL_MEM_FENCE); // sync group

  // Reduce sums in log2(cols) steps
  while ( cols > 1 )
    {
      cols >>= 1;
      if (jj < cols) work[ii+rows*jj] += work[ii+rows*(jj+cols)];
      barrier(CLK_LOCAL_MEM_FENCE); // sync group
    }

  // Write final result in Y
  if ( jj == 0 ) y[get_global_id(ROW_DIM)] = work[ii];
}
"""

In [14]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [15]:
program = cl.Program(context, gpu_matmul).build()


# Initialise data
offset = np.average(y, weights = w)
g = y-offset
mstop = 100
lr = 0.1 # Learning rate

final_coefs = [np.zeros(X.shape[1], dtype = np.float32) for X in design_matrices]
delta = [np.zeros(X.shape[1], dtype = np.float32) for X in design_matrices]
eta = [np.zeros(X.shape[0], dtype = np.float32) for X in design_matrices]
rss = np.zeros(len(design_matrices), dtype = np.float32)
benchmark = np.empty((mstop, n_learners))
benchmark_GPU = np.empty((mstop, n_learners))

In [16]:
# This is the part we need to have in opencl
for m in range(mstop):
  Wg = g*w
  
  # Expensive part of the function
  for i in range(n_learners):
  # for i in range(1):
    # Per learner iterations
    A = multiplicands_for_beta[i]
    X = design_matrices[i]
    
    #== Compute runtime
    start = time.time()
 
    # Matrix multiplications
    XtWg = X.T @ Wg # O(np)
    # print(np.shape(X.T), np.shape(Wg))
    # print(np.shape(XtWg))
    delta[i] = lr*(A @ XtWg) # O(p^2) 
    
    eta[i] = X @ delta[i] # O(np)
    
    #== Compute runtime
    end = time.time()
    t = end - start
    benchmark[m,i]=t
    
    rss[i] = np.sum((g-eta[i])**2)

  ind = np.argmin(rss)
  # Update model
  # print(np.shape(delta[ind]))
  final_coefs[ind] += delta[ind]
  g -= eta[ind]

  benchmark.mean()
  benchmark_GPU.mean()

In [18]:
# This is the part we need to have in opencl
for m in range(mstop):
  Wg = np.asarray((g*w)).astype(np.float32)
  
  # Expensive part of the function
  for i in range(n_learners):
  # for i in range(1):
    # Per learner iterations
    A = multiplicands_for_beta[i].astype(np.float32)
    X = design_matrices[i].astype(np.float32)
    
    #== Compute runtime
    # start = time.time()
 
    # Matrix multiplications
    # XtWg = X.T @ Wg # O(np)
    a = (X.T).astype(np.float32)
    b = Wg
    m, n = np.shape(a)
    k = 1
    XtWg = np.empty(m).astype(np.float32)

    mf = cl.mem_flags
    a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
    b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)
    c_buf = cl.Buffer(context, mf.WRITE_ONLY, XtWg.nbytes)

    wk = 16
    TS = 16
    local = (TS, 1)
    # local_buf_size = local[0]*local[1]*np.int32(4)
    global_size = (np.int32(m), np.int32(k))

    # local = tuple(map(int, local))
    # global_size = tuple(map(int, global_size))


    # kernel = program.matvecmul
    kernel = program.gemv2
    kernel.set_arg(0, np.int32(m))
    kernel.set_arg(1, np.int32(n))
    kernel.set_arg(2, a_buf)
    kernel.set_arg(3, b_buf)
    kernel.set_arg(4, c_buf)
    # kernel.set_arg(5, local_buf_size)
    
    # kernel.set_arg(5, work_buf)
    
    # kernel.set_arg(4, np.int32(k))
    # kernel.set_arg(5, c_buf)

    event = cl.enqueue_nd_range_kernel(queue, kernel, global_size, local)

    # event = program.matmul(queue, global_size, local,
    #                         np.int32(a.shape[1]), np.int32(b.shape[1]), np.int32(b.shape[0]),
    #                         a_buf, b_buf, c_buf)

    event.wait()
    cl.enqueue_copy(queue, XtWg, c_buf)
    a = A
    b = XtWg
    m, n = np.shape(a)
    k = 1
    elta = np.empty(m).astype(np.float32)
    mf = cl.mem_flags
    a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
    b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)
    c_buf = cl.Buffer(context, mf.WRITE_ONLY, elta.nbytes)


    wk = 16
    TS = 16
    local = (TS, 1)
    # local_buf_size = local[0]*local[1]*np.int32(4)
    global_size = (np.int32(m), np.int32(k))

    # local = tuple(map(int, local))
    # global_size = tuple(map(int, global_size))


    # kernel = program.matvecmul
    kernel = program.gemv2
    kernel.set_arg(0, np.int32(m))
    kernel.set_arg(1, np.int32(n))
    kernel.set_arg(2, a_buf)
    kernel.set_arg(3, b_buf)
    kernel.set_arg(4, c_buf)
    # kernel.set_arg(5, local_buf_size)
    # kernel.set_arg(5, work_buf)
    
    # kernel.set_arg(4, np.int32(k))
    # kernel.set_arg(5, c_buf)

    event = cl.enqueue_nd_range_kernel(queue, kernel, global_size, local)

    # event = program.matmul(queue, global_size, local,
    #                         np.int32(a.shape[1]), np.int32(b.shape[1]), np.int32(b.shape[0]),
    #                         a_buf, b_buf, c_buf)

    event.wait()
    cl.enqueue_copy(queue, elta, c_buf)

    # delta[i] = lr*(A @ XtWg) #.flatten() # O(p^2) 
    delta[i] = lr * elta
    a = X
    b = delta[i]
    m, n = np.shape(a)
    k = 1
    ta = np.empty(m).astype(np.float32)
    mf = cl.mem_flags
    a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
    b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)
    c_buf = cl.Buffer(context, mf.WRITE_ONLY, ta.nbytes)

    TS = 16
    local_size = (1, TS)
    # local_buf_size = np.int32(local[0]*local[1])
    global_size = (np.int32(m), np.int32(k))
    if global_size[0] % local_size[0] != 0:
      global_size = (global_size[0] + local_size[0] - global_size[0] % local_size[0], global_size[1])
    if global_size[1] % local_size[1] != 0:
      global_size = (global_size[0], global_size[1] + local_size[1] - global_size[1] % local_size[1])

    # local = tuple(map(int, local))
    # global_size = tuple(map(int, global_size))


    # kernel = program.matvecmul
    kernel = program.gemv3
    kernel.set_arg(0, np.int32(m))
    kernel.set_arg(1, np.int32(n))
    kernel.set_arg(2, a_buf)
    kernel.set_arg(3, b_buf)
    kernel.set_arg(4, c_buf)
    # kernel.set_arg(5, local_buf_size)

    event = cl.enqueue_nd_range_kernel(queue, kernel, global_size, local_size)

    # event.wait()
    # cl.enqueue_copy(queue, ta, c_buf)
    # eta[i] = X @ delta[i]
    eta[i] = ta # O(np)
    
    rss[i] = np.sum((g-eta[i])**2)

  ind = np.argmin(rss)
  # Update model
  # print(np.shape(delta[ind]))
  final_coefs[ind] += delta[ind]
  g -= eta[ind]

  # benchmark.mean()
  benchmark_GPU.mean()

In [225]:
np.shape(X)

(1000, 32)

In [62]:
gpu_matmul = """
#define TSM 128                // The tile-size in dimension M
#define TSN 128                // The tile-size in dimension N
#define TSK 16                 // The tile-size in dimension K
#define WPTM 8                 // The work-per-thread in dimension M
#define WPTN 8                 // The work-per-thread in dimension N
#define RTSM (TSM/WPTM)        // The reduced tile-size in dimension M
#define RTSN (TSN/WPTN)        // The reduced tile-size in dimension N
#define LPTA ((TSK*TSM)/(RTSM*RTSN)) // Loads-per-thread for A
#define LPTB ((TSK*TSN)/(RTSM*RTSN)) // Loads-per-thread for B

// Use 2D register blocking (further increase in work per thread)
__kernel void matmul(const int M, const int N, const int K,
                      const __global float* A,
                      const __global float* B,
                      __global float* C) {
    
    // Thread identifiers
    const int tidm = get_local_id(0); // Local row ID (max: TSM/WPTM)
    const int tidn = get_local_id(1); // Local col ID (max: TSN/WPTN)
    const int offsetM = TSM*get_group_id(0); // Work-group offset
    const int offsetN = TSN*get_group_id(1); // Work-group offset
 
    // Local memory to fit a tile of A and B
    __local float Asub[TSK][TSM];
    __local float Bsub[TSN][TSK+2];
 
    // Allocate register space
    float Areg;
    float Breg[WPTN];
    float acc[WPTM][WPTN];
 
    // Initialise the accumulation registers
    for (int wm=0; wm<WPTM; wm++) {
        for (int wn=0; wn<WPTN; wn++) {
            acc[wm][wn] = 0.0f;
        }
    }
    
    // Loop over all tiles
    int numTiles = K/TSK;
    for (int t=0; t<numTiles; t++) {
 
        // Load one tile of A and B into local memory
        for (int la=0; la<LPTA; la++) {
            int tid = tidn*RTSM + tidm;
            int id = la*RTSN*RTSM + tid;
            int row = id % TSM;
            int col = id / TSM;
            int tiledIndex = TSK*t + col;
            Asub[col][row] = A[tiledIndex*M + offsetM + row];
            Bsub[row][col] = B[tiledIndex*N + offsetN + row];
        }
        
        // Synchronise to make sure the tile is loaded
        barrier(CLK_LOCAL_MEM_FENCE);
 
        // Loop over the values of a single tile
        for (int k=0; k<TSK; k++) {
 
            // Cache the values of Bsub in registers
            for (int wn=0; wn<WPTN; wn++) {
                int col = tidn + wn*RTSN;
                Breg[wn] = Bsub[col][k];
            }
 
            // Perform the computation
            for (int wm=0; wm<WPTM; wm++) {
                int row = tidm + wm*RTSM;
                Areg = Asub[k][row];
                for (int wn=0; wn<WPTN; wn++) {
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
 
        // Synchronise before loading the next tile
        barrier(CLK_LOCAL_MEM_FENCE);
    }
 
    // Store the final results in C
    for (int wm=0; wm<WPTM; wm++) {
        int globalRow = offsetM + tidm + wm*RTSM;
        for (int wn=0; wn<WPTN; wn++) {
            int globalCol = offsetN + tidn + wn*RTSN;
            C[globalCol*M + globalRow] = acc[wm][wn];
        }
    }
}
"""

In [5]:
gpu_matmul_global = """
#pragma OPENCL EXTENSION cl_khr_fp64: enable

__kernel void matmul(__global float* C, 
          __global float* A, 
          __global float* B, 
          const int wA, const int wB){
  
   int tile_x = get_global_id(0); 
   int tile_y = get_global_id(1);
 
   // value stores the element that is computed by the thread
   float value = 0;
   for (int k = 0; k < wA; ++k){
      float elementA = A[tile_y * wA + k];
      float elementB = B[k * wB + tile_x];
      value += elementA * elementB;

      barrier(CLK_LOCAL_MEM_FENCE);
   }
 
   // Write the matrix to device memory each thread writes one element
   C[tile_y * wA + tile_x] = value;
}
"""

In [7]:
n = np.int32(2**12)
m = n
k = n

a = np.random.rand(m, n).astype(np.float32)
b = np.random.rand(n, k).astype(np.float32)
c = np.empty_like(np.random.rand(m,k).astype(np.float32))

mf = cl.mem_flags
a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)
c_buf = cl.Buffer(context, mf.WRITE_ONLY, c.nbytes)


TS = 16
local = (TS, TS)
global_size = (m, n)

# local = tuple(map(int, local))
# global_size = tuple(map(int, global_size))


kernel = program.matmul

kernel.set_arg(0, c_buf)
kernel.set_arg(1, a_buf)
kernel.set_arg(2, b_buf)
kernel.set_arg(3, m)
kernel.set_arg(4, k)
# kernel.set_arg(5, c_buf)

event = cl.enqueue_nd_range_kernel(queue, kernel, global_size, local)

# event = program.matmul(queue, global_size, local,
#                         np.int32(a.shape[1]), np.int32(b.shape[1]), np.int32(b.shape[0]),
#                         a_buf, b_buf, c_buf)

event.wait()


In [8]:
cl.enqueue_copy(queue, c, c_buf)
c

array([[1008.2011 , 1021.35754, 1018.945  , ..., 1024.4844 , 1006.6604 ,
        1017.56805],
       [1016.3794 , 1021.7331 , 1018.2468 , ..., 1010.4697 , 1010.12665,
        1020.1729 ],
       [1019.67413, 1030.9456 , 1030.3876 , ..., 1035.0476 , 1022.8908 ,
        1023.09827],
       ...,
       [1014.7698 , 1012.31635, 1008.79425, ..., 1013.1611 , 1007.46075,
        1020.8824 ],
       [1020.79803, 1023.55304, 1018.0205 , ..., 1026.9062 , 1009.5984 ,
        1019.94403],
       [1010.64594, 1008.86414, 1014.6246 , ..., 1019.57306, 1021.329  ,
        1008.7856 ]], dtype=float32)

In [ ]:
print(np.dot(a,b))
print(np.matmul(a,b))

In [15]:
np.testing.assert_almost_equal(np.matmul(a,b), c, decimal=2)

In [20]:
gpu_matmul = """
#define BLOCK_SIZE 16
#define global_idx(x_idx, y_idx, m) (x_idx * m + y_idx)

#define WPT 8
#define RBLOCK_SIZE (BLOCK_SIZE/WPT)

// -- Uses NDRange Kernel with Local Memory and 2D Register tiling
// M, N, P can be arbitrary sizes
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE / WPT, BLOCK_SIZE / WPT, 1)))
void matmul(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P,
      __const int M_,
      __const int N_,
      __const int P_)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float Areg;
    float Breg[WPT];
    float acc[WPT][WPT];
    for(int wm=0; wm<WPT; wm++){
        for(int wn=0; wn<WPT; wn++){
            acc[wm][wn] = 0.0f;
        }
    }
    const int numTiles = N_/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int wm=0; wm<WPT; wm++){
            for (int wn=0; wn<WPT; wn++){
                const int r = BLOCK_SIZE*t + row;
                const int c = BLOCK_SIZE*t + col;
                if(((m + wm*RBLOCK_SIZE) < M) && ((c + wn*RBLOCK_SIZE) < N)){
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = A[(m + wm*RBLOCK_SIZE)*N + (c + wn*RBLOCK_SIZE)];
                } else {
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }

                if(((p + wn*RBLOCK_SIZE) < P) && ((r + wm*RBLOCK_SIZE) < N)){
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = B[(r + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)];
                } else {
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int wn=0; wn<WPT; wn++){
                Breg[wn] = B_local[k][col + wn*RBLOCK_SIZE];
            }
            for (int wm=0; wm<WPT; wm++){
                Areg = A_local[row + wm*RBLOCK_SIZE][k];
                for (int wn=0; wn<WPT; wn++){
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int wm=0; wm<WPT; wm++){
        for (int wn=0; wn<WPT; wn++){
            if(((m + wm*RBLOCK_SIZE) < M) && ((p + wn*RBLOCK_SIZE) < P)){
                C[(m + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)] = acc[wm][wn];
            }
            
        }
    }
}
"""

In [21]:
# os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'
# os.environ['PYOPENCL_CTX'] = '0:1'

class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start
        
def roundUp(numToRound, multiple):
    if (multiple == 0):
        return numToRound
    remainder = numToRound % multiple
    if (remainder == 0):
        return numToRound
    return numToRound + multiple - remainder
BLOCK_SIZE = 32
WPT = 8

mult = 3
m, n, k = 3*(10**mult), 4*(10**mult), 5*(10**mult)
m_, n_, k_ = roundUp(m, BLOCK_SIZE), roundUp(n, BLOCK_SIZE), roundUp(k, BLOCK_SIZE)
a = np.random.randn(m, n).astype(np.float32)
b = np.random.randn(n, k).astype(np.float32)


program = cl.Program(context, gpu_matmul).build()

In [22]:
np_times, py_times, cl_times = [], [], []

# ctx = cl.create_some_context()
# queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = a)
b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = b)

start = time.time()
c = a.dot(b)
end = time.time()
print('np: ', end - start)
np_times.append(end - start)
start = time.time()
c = a @ b
end = time.time()
print('py: ', end - start)
py_times.append(end - start)

c_res = np.zeros_like(c)
c_buf = cl.Buffer(context, mf.WRITE_ONLY, c_res.nbytes)
c.shape

with Timer() as t:
    event = program.matmul(queue, (m_ // WPT , k_ // WPT), (BLOCK_SIZE // WPT, BLOCK_SIZE // WPT), 
                     a_buf, b_buf, c_buf,
                     np.int32(m), np.int32(n), np.int32(k),
                     np.int32(m_), np.int32(n_), np.int32(k_))
    event.wait()
print('cl: ', t.interval)
cl_times.append(t.interval)
cl.enqueue_copy(queue, c_res, c_buf)

c_res.shape, m, n, k

np:  0.3036017417907715
py:  0.29758572578430176
cl:  0.5545623302459717


((3000, 5000), 3000, 4000, 5000)

In [19]:
np.testing.assert_almost_equal(c_res, c, decimal=3)